针对上述任意样本，比如SGI75460067的count/tpm/cpm，会得到如下的结果
|基因名称|	Hisat-rf结果|	Hisat-fr结果|	Star结果|
|-|-|-|-|
|ABC	|150	|140	|600|
|DEF	|120000	|10000	|2600|
|GHI	|0	|1|200|

算了，算了，还是手动合并算了，不行啊 还是要写代码

In [ ]:
# import os
# import pandas as pd
# #"D:\\githubku\\my_jupyter\\20230206_tpm_count\\fina/12"文件夹下所有文件夹名，保存为list
# path = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/12"
# file_list = os.listdir(path)
# # 读取D:\\githubku\\my_jupyter\\20230206_tpm_count\\final中的count_all.xlsx，cpm_all.xlsx，tpm_all.xlsx
# count_all = pd.read_excel("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/count_all.xlsx")
# cpm_all = pd.read_excel("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/cpm_all.xlsx")
# tpm_all = pd.read_excel("D:\\githubku\\my_jupyter\\20230206_tpm_count\\final/tpm_all.xlsx")

In [118]:
import pandas as pd
import os
ref = pd.read_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\/merge1v1.txt",sep="\t")
##每种类型所包含的基因名称，以及

C:\Users\zhangyifan1\AppData\Local\Temp\ipykernel_27100\1681191210.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  ref = pd.read_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\/merge1v1.txt",sep="\t")


In [140]:
id="SGI75460067"

In [141]:
# D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\type_RF_star\\tpm\\tpm

In [142]:
inputpath1 = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\sample_RF_star\\cpm\\cpm"+id+".txt"
outputpath1="D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\type_RF_star\\cpm\\"+id+"\\"

In [143]:
os.makedirs(os.path.dirname(outputpath1), exist_ok=True)

In [144]:
#保留Gene Name	Gene type列
ref = ref[["Gene Name","Gene type"]]
#去除gene name重复行
ref = ref.drop_duplicates(subset="Gene Name",keep="first")

In [145]:
#读入D:\githubku\my_jupyter\20230206_tpm_count\final\12\SGI75460067\0067count.txt文件
count = pd.read_csv(inputpath1,sep="\t")

In [146]:
#ref和count合并，以gene name为键，保留ref中的gene type
count1 = pd.merge(ref,count,left_on="Gene Name",right_on="Gene Name_RFfr",how="right")


In [147]:
count1

,Gene Name,Gene type,Unnamed: 0,Unnamed: 1,Gene Name_RFfr,count_RFfr,cpm_RFfr,Gene Name_star,count_star,cpm_star
0,A1BG,protein_coding,SGI75460067,45088,A1BG,5,0.505138,A1BG,9,1.319955
1,A1BG-AS1,lncRNA,SGI75460067,45087,A1BG-AS1,0,0.000000,A1BG-AS1,0,0.000000
2,A1CF,protein_coding,SGI75460067,45086,A1CF,5,0.505138,A1CF,19,2.786571
3,A2M,protein_coding,SGI75460067,45085,A2M,245,24.751743,A2M,221,32.412220
4,A2M-AS1,lncRNA,SGI75460067,45084,A2M-AS1,3,0.303083,A2M-AS1,3,0.439985
...,...,...,...,...,...,...,...,...,...,...
45084,ZYX,protein_coding,SGI75460067,4,ZYX,126,12.729468,ZYX,35,5.133157
45085,ZYXP1,pseudo,SGI75460067,3,ZYXP1,0,0.000000,ZYXP1,0,0.000000
45086,ZZEF1,protein_coding,SGI75460067,2,ZZEF1,84,8.486312,ZZEF1,71,10.412976
45087,ZZZ3,protein_coding,SGI75460067,1,ZZZ3,45,4.546238,ZZZ3,43,6.306450


In [148]:
#将count1中的gene type列，转换为行索引
count1 = count1.set_index("Gene type")


In [149]:
#输出count1中gene type为空的行
count1.loc[count1.index.isnull()]

,Gene Name,Unnamed: 0,Unnamed: 1,Gene Name_RFfr,count_RFfr,cpm_RFfr,Gene Name_star,count_star,cpm_star
Gene type,,,,,,,,,


In [150]:
#获取count1中索引，保存为list，去重
index_name = list(set(count1.index))
#遍历index_name,获取count1中对应索引的数据，保存到文件中
for i in index_name:
    count1[count1.index==i].to_csv(outputpath1+"\\"+id+".%s.txt"%i,sep="\t")


## 计算pearson相关系数和p     使用r语言

In [153]:
#创建一个空的数据框
count_all = pd.DataFrame()
#遍历file_list，读取文件，保存到count_all中,文件名作为索引
file_list = os.listdir("D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\type_RF_star\\sum\\tpm")
for i in file_list:
    file = pd.read_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\type_RF_star\\sum\\tpm\\"+i,sep=",",index_col=0)
    #将文件名设置为多级索引
    #删除i包含的sum.txt
    i = i.replace("sum.txt","")
    file.index = pd.MultiIndex.from_product([[i],file.index])
    #将file添加到count_all中
    count_all = pd.concat([count_all,file],axis=0)
count_all
    

cpm_RFfr:cpm_star_r  cpm_RFfr:cpm_star_p
SGI75460067 Mt_rRNA                   NaN                  NaN
            Mt_tRNA              0.948926         1.764397e-11
            lncRNA               0.746523         0.000000e+00
            miRNA                0.919610         0.000000e+00
            misc_RNA             0.961109         5.610211e-04
...                                   ...                  ...
SGI75460105 rRNA                -0.142828         4.598329e-01
            scRNA                0.677005         3.229954e-01
            snRNA                0.130272         1.145372e-01
            snoRNA               0.930329         0.000000e+00
            tRNA                 0.989180         0.000000e+00

[91 rows x 2 columns]

In [154]:
# 创建一个空的数据框
count_all1 = pd.DataFrame()
count_temp = pd.DataFrame()
#在多级索引中分别查找"Mt_rRNA", "Mt_tRNA", "lncRNA", "miRNA", "misc_RNA", "other", "protein_coding", "pseudo", "rRNA", "scRNA", "snRNA", "snoRNA", "tRNA"的数据
for i in ["Mt_rRNA", "Mt_tRNA", "lncRNA", "miRNA", "misc_RNA", "other", "protein_coding", "pseudo", "rRNA", "scRNA", "snRNA", "snoRNA", "tRNA"]:
    count_temp=count_all.loc[count_all.index.get_level_values(1)==i]
    # 删除count_temp中第一个索引
    count_temp = count_temp.droplevel(1)
    # count_temp设置为多级索引
    count_temp.index = pd.MultiIndex.from_product([[i],count_temp.index])
    count_all1 = pd.concat([count_all1,count_temp],axis=0)
# count_all1输出到文件中
count_all1.to_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\type_RF_star\\sum\\final\\tpm_all.txt",sep="\t")


## 合并tpm count cpm

In [53]:
import pandas as pd
import os
#合并cpm 和 tpm
# 创建空dataframe
cpm_1 = pd.DataFrame()
cpm_2 = pd.DataFrame()
path_filefolder1 = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\RFfr\\tpm"
path_filefolder2="D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\star\\tpm"
outputfilefolder = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\RF_star\\tpm"
#遍历读取D:\githubku\my_jupyter\20230206_tpm_count\0215cpm_tpm_pearson\cpm_tpm\RFfr\cpm中的文件
file_list1 = os.listdir(path_filefolder1)
for i1 in file_list1:
    file1 = pd.read_csv(path_filefolder1+"\\"+i1,sep=",")
    #按照Gene Name升序排列
    file1 = file1.sort_values(by="Gene Name")
    #将文件名设置为多级索引
    #删除i包含的sum.txt
    i1 = i1.replace("Name_tPM.csv","")
    file1.index = pd.MultiIndex.from_product([[i1],file1.index])
    #将file添加到count_all中
    cpm_1 = pd.concat([cpm_1,file1],axis=0)
#遍历读取D:\githubku\my_jupyter\20230206_tpm_count\0215cpm_tpm_pearson\cpm_tpm\star\cpm中的文件
file_list2 = os.listdir(path_filefolder2)
for i2 in file_list2:
    file2 = pd.read_csv(path_filefolder2+"\\"+i2,sep=",")
    #将文件名设置为多级索引
    file2 = file2.sort_values(by="Gene Name")
    #删除i包含的sum.txt
    i2 = i2.replace("Name_tPM.csv","")
    file2.index = pd.MultiIndex.from_product([[i2],file2.index])
    #将file添加到count_all中
    cpm_2 = pd.concat([cpm_2,file2],axis=0)
    
        

In [54]:
# cpm_1列名修改为Gene Name_RFfr,tpm_RFfr
cpm_1.columns = ["Gene Name_RFfr","tpm_RFfr"]
# cpm_2列名修改为Gene Name_star,tpm_star
cpm_2.columns = ["Gene Name_star","tpm_star"]

In [ ]:
# cpm_1列名修改为Gene Name_RFfr,count_RFfr,cpm_RFfr
cpm_1.columns = ["Gene Name_RFfr","count_RFfr","cpm_RFfr"]
# cpm_2列名修改为Gene Name_star,count_star,cpm_star
cpm_2.columns = ["Gene Name_star","count_star","cpm_star"]

In [56]:
#将cpm_1和cpm_2按照level_values(1)合并
cpm_12 = cpm_1.join(cpm_2)

In [ ]:
cpm_12.loc[cpm_12.index.get_level_values(0)==i]

In [58]:
# 输出cpm_12以及索引对应的数据
# 创建一个空的数据框
cpm_12_1 = pd.DataFrame()
id = cpm_12.index.get_level_values(0)
#遍历cpm_12的索引，获取对应的数据，保存到cpm_12_1中，依次输出
for i in set(cpm_12.index.get_level_values(0)):
    cpm_12_1 = cpm_12.loc[cpm_12.index.get_level_values(0)==i]
    cpm_12_1.to_csv("D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\sample_RF_star\\tpm\\tpm"+i+".txt",sep="\t")

In [ ]:

import pandas as pd
import os
## 合并同一批样本的tpm  
inputfilefolder = "D:\\githubku\\my_jupyter\\20230206_tpm_count\\0215cpm_tpm_pearson\\cpm_tpm\\RFfr\\tpm"
#遍历文件夹的文件，将文件名保存到file_list中
file_list = os.listdir(inputfilefolder)
# 创建空dataframe
tpm_1 = pd.DataFrame()
#遍历读取D:\githubku\my_jupyter\20230206_tpm_count\0215cpm_tpm_pearson\cpm_tpm\RFfr\tpm中的文件,按照Gene Name合并
for i in file_list:
    file = pd.read_csv(inputfilefolder+"\\"+i,sep=",")
    #按照Gene Name升序排列
    file = file.sort_values(by="Gene Name")
    #文件名替换TPM作为列名
    i = i.replace("Name_tPM.csv","")
    file.columns = ["Gene Name",i]
    #依次将file添加到tpm_1中
    tpm_1 = pd.concat([tpm_1,file],axis=0)
#输出tpm_1
tpm_1

In [9]:
import pandas as pd
import os
## 合并同一批样本的tpm  
inputfilefolder = "C:\\Users\\zhangyifan1\\Desktop\\0216\\0210\\name_tpm\\"
outputfile= "C:\\Users\\zhangyifan1\\Desktop\\name_tpm\\0210name_tpm.csv"
#遍历文件夹的文件，将文件名保存到file_list中
file_list = os.listdir(inputfilefolder)
# 创建空dataframe
tpm_1 = pd.DataFrame()
cnt=1
#遍历读取D:\githubku\my_jupyter\20230206_tpm_count\0215cpm_tpm_pearson\cpm_tpm\RFfr\tpm中的文件,按照Gene Name合并
#第一个文件保存为file，之后的文件保存为file2
for i in file_list:
    if cnt ==1:
        file = pd.read_csv(inputfilefolder+"\\"+i,sep=",")
        #按照Gene Name升序排列
        file = file.sort_values(by="Gene Name")
        #文件名替换TPM作为列名
        i = i.replace("Name_tPM.csv","")
        file.columns = ["Gene Name",i]
        #依次将file横向添加到tpm_1中
        tpm_1 = pd.concat([tpm_1,file],axis=1)
        #将tpm_1第一列设为索引
        tpm_1 = tpm_1.set_index("Gene Name")
        cnt +=1
    else:
        file2 = pd.read_csv(inputfilefolder+"\\"+i,sep=",")
        #按照Gene Name升序排列
        file2 = file.sort_values(by="Gene Name")
        #文件名替换TPM作为列名
        i = i.replace("Name_tPM.csv","")
        file2.columns = ["Gene Name",i]
        # 将file2合并到tpm_1中
        tpm_1 = pd.concat([tpm_1,file2[i]],axis=1)
        cnt+=1
tpm_1.to_csv(outputfile,sep=",")
    